<a href="https://colab.research.google.com/github/fanny/information-retrieval/blob/master/lab5/model_vectorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re

import pandas as pd
import numpy as np

import time

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

from collections import defaultdict
from math import log

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('https://raw.githubusercontent.com/fanny/information-retrieval/master/lab5/results.csv')
data.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...
5,Rosângela uma das crianças sequestradas por f...,“Roubaram minha identidade” diz Rosângela Par...,Joana Oliveira,30/03/2019 22:21:08,Brasil,Rosângela Serra Paraná não sabe quantos anos t...,https://brasil.elpais.com/brasil/2019/03/29/po...
6,“Lógica de usar torturadores da ditadura no cr...,Aloy Jupiara coautor de 'Os porões da contrav...,Felipe Betim,01/04/2019 13:22:17,Brasil,Trajetória similar tiveram outros vários agent...,https://brasil.elpais.com/brasil/2019/03/29/po...
7,Bolsonaro troca embaixada por escritório em Je...,Autoridade Palestina convoca embaixador no Bra...,Juan Carlos Sanz,01/04/2019 12:26:32,Brasil,A devoção dos cristãos evangélicos em seu apoi...,https://brasil.elpais.com/brasil/2019/03/31/po...
8,O lado mais sombrio de Dickens,Escritor e jornalista tentou internar sua mulh...,Rafa de Miguel,10/03/2019 14:29:37,Cultura,O homem mais famoso da era vitoriana o “poeta...,https://brasil.elpais.com/brasil/2019/03/01/cu...
9,O amor entre mulheres que sacudiu o Paraguai,O drama ganhou três prêmios em Berlim e provoc...,Gregorio Belinchón,09/03/2019 12:10:23,Cultura,Depois de inclusive o da crítica internacion...,https://brasil.elpais.com/brasil/2019/03/07/cu...


In [0]:
clean_text = data['text'].dropna()
data['_id'] = list(data.index)

In [0]:

def tokenize_text(text):
  hyphenated_and_abreviated_words = '\w{2,}[\-|\.]*\w+'
  decimal_numbers_with_symbols = '\w?\W?\d{2,}[\.|\,]*\d+\W?'
  words ='\w{2,}'
  numbers_with_symbols = '\w?\W?\d{2,}\W?'
  pattern = '|'.join([hyphenated_and_abreviated_words, decimal_numbers_with_symbols, words, numbers_with_symbols])

  word_list = regexp_tokenize(text, pattern=pattern)
  return word_list

In [0]:
tokens_per_document = clean_text.apply(tokenize_text)
tokens_per_document_dict = {'docid': tokens_per_document.index.tolist(), 'token': tokens_per_document.values}
tokens_per_document = pd.DataFrame(tokens_per_document_dict)
tokens_per_document.head()

,docid,token
0,0,"[juíza, federal, Ivani, Silva, da, Luz, de, Br..."
1,1,"[Menos, de, 24 , horas, depois, de, juíza, fe..."
2,2,"[Quando, determinou, que, de, 31 , de, março,..."
3,3,"[Quando, era, pequeno, Luisa, Gabriel, se, pre..."
4,4,"[Desde, manhã, da, última, segunda-feira, suce..."


## IDF Functions

In [0]:
def get_total_frequecy_of_terms():
  total_frequency_of_terms = defaultdict(int)
  
  for index, row in tokens_per_document.iterrows():
    values = row['token']
    for value in values:
      if value:
        total_frequency_of_terms[value] += 1
  
  return total_frequency_of_terms

In [0]:
N = len(tokens_per_document)

def compute_idf():
  idf_dict = defaultdict(int)
  for token, freq in get_total_frequecy_of_terms().items():
    idf_dict[token] = log(N/freq)
        
        
  return idf_dict
    

## Utilitary Functions

In [0]:
idf = compute_idf()
def has_key(the_dict, key, idf_token):
  return the_dict[(key, idf_token)]

def create_new_record(docid):
  return (docid, 1)

def search_docid_record_in_indexes(values, docid):
  index = -1
  count = 0
  
  for (i, value) in enumerate(values):
    tuple_docid, frequency = value
    if tuple_docid == docid:
      index = i
      count = frequency
      
  return (index, count)

In [0]:
indexes = defaultdict(list)


def build_index(row):
  docid, tokens = row
  for token in tokens:
    idf_token = idf[token]
    if has_key(indexes, token, idf_token):
      values = indexes[(token, idf_token)]
      index_of_record, frequency = search_docid_record_in_indexes(values, docid)
      is_encontered = index_of_record != -1 
      if is_encontered:
        indexes[(token, idf_token)][index_of_record] = (docid, frequency + 1)
      else:
        indexes[(token, idf_token)].append(create_new_record(docid))
    else:
        indexes[(token, idf_token)].append(create_new_record(docid))
        
        
tokens_per_document.apply(build_index, axis=1)
len(indexes)

26856

## Binary Representation

In [0]:
def build_binary_representation_doc(query_terms, doc_terms):
  vector_len = len(query_terms)
  doc_binary_rep = np.zeros(vector_len)
  for index, query_term in enumerate(query_terms):
    doc_terms = list(map(str.lower, doc_terms))
    if query_term in doc_terms:
      doc_binary_rep[index] = 1
      
  return doc_binary_rep
      

      
def get_binary_representation_doc(query):
  query_terms = query.lower().split(' ')
  doc_binary_rep = defaultdict(list)
  for index, row in tokens_per_document.iterrows():
    docid = row['docid']
    values = row['token']
    doc_binary_rep[docid] = build_binary_representation_doc(query_terms, values)
      
  return doc_binary_rep
    
    


In [0]:
def get_binary_similarity(binary_representation_doc):
  doc_similarity = {}
  for doc, vector in binary_representation_doc.items():
    doc_similarity[doc] = sum(vector)
    
  return doc_similarity

## Term Frequency Representation


In [0]:
def build_tf_representation_doc(query):
  tf_representation = {}
  query_terms = query.lower().split(' ')
  for query_term in query_terms:
    idf_token = idf[query_term]
    for the_tuple in indexes[(query_term, idf_token)]:
      docid, frequency = the_tuple
      tf_representation[(docid, query_term)] = frequency
   
  return tf_representation
    


## TF-IDF Representation

In [0]:
def build_tf_idf_representation(query):
  tf_idf_representation = {}
  tf_representation = build_tf_representation_doc(query)
  tokens = query.lower().split(' ')
  for tf_index in tf_representation:
    _, term = tf_index
    tf_value = tf_representation[tf_index]
    tf_idf_representation[tf_index] = tf_value * idf[term]
    
  return tf_idf_representation




## BM25

In [0]:
def get_n(query_term, tf_representation):
  count = 0
  for index in tf_representation:
    _, term = index
    if term.lower() == query_term.lower():
      count += 1
      
  return count

In [0]:
def build_bm25_representation(query):
  bm25_representation = {}
  k = 2
  b = 0.75
  len_data = len(data)
  tf_representation = build_tf_representation_doc(query)
  tokens = query.lower().split(' ')
  for tf_index in tf_representation:
    _, term = tf_index
    tf_value = tf_representation[tf_index]
    n = get_n(term, tf_representation)
    f4 = n and log((len_data - n)/n)
    bm25_representation[tf_index] = ((tf_value * (k+1))/(tf_value + k)) * f4
    
  return bm25_representation

## Testes Binary Representation


In [0]:
def get_bin_ranking(query):
  binary_representation_doc = get_binary_representation_doc(query)
  binary_similarity = get_binary_similarity(binary_representation_doc)
  binary_similarity = pd.DataFrame(binary_similarity.values(), index=binary_similarity.keys(), columns=['bin_similarity'])
  ranking_binary = binary_similarity.sort_values(by='bin_similarity', ascending=False)
  return ranking_binary.head()

In [0]:
def get_tf_ranking(query):
  tf_rep = build_tf_representation_doc(query)
  docids = [docid for docid, _ in tf_rep.keys()]
  values = tf_rep.values()
  ranking = pd.DataFrame(values, columns=['tf_score'], index=docids)
  ranking = ranking.sort_values(by='tf_score', ascending=False)
  return ranking.head()

In [0]:
def get_tf_idf_ranking(query):
  tf_idf_rep = build_tf_idf_representation(query)
  docids = [docid for docid, _ in tf_idf_rep.keys()]
  values = tf_idf_rep.values()
  ranking = pd.DataFrame(values, columns=['tf_idf_score'], index=docids)
  ranking = ranking.sort_values(by='tf_idf_score', ascending=False)
  return ranking.head()


In [0]:
def get_bm25_ranking(query):
  bm25_rep = build_bm25_representation(query)
  docids = [docid for docid, _ in bm25_rep.keys()]
  values = bm25_rep.values()
  ranking = pd.DataFrame(values, columns=['bm25_score'], index=docids)
  ranking = ranking.sort_values(by='bm25_score', ascending=False)
  return ranking.head()



## Query 1

In [0]:
results_table1 = {
    'Binary similarity': get_bin_ranking('juíza federal').index,
    'TF similarity': get_tf_ranking('juíza federal').index,
    'TF IDF similarity': get_tf_idf_ranking('juíza federal').index,
    'Bm25 similarity': get_bm25_ranking('juíza federal').index
}
pd.DataFrame(results_table1)

,Binary similarity,TF similarity,TF IDF similarity,Bm25 similarity
0,0,150,0,0
1,1,0,150,1
2,207,6,1,150
3,222,36,6,6
4,33,212,36,36


Os resultados mais satisfatórios são provenientes da similaridade binária, analisando as notícias, da mesma, vemos que todas possuem uma relação com julgamentos, enquanto que nas outras abordagens estão muito mais relacionadas a nóticias de economia do governo do atual presidente, Jair Bolsonaro. Podemos concluir que isso se dá ao fato de que, os outros métodos, por considerarem a frequência, e por terem 'federal' como um termo bastante comum, acabam colocando esses documentos em posições mais elevadas, enquanto que no caso da similaridade binária, por não considerar relevante a frequência, considera mais documentos que contém ambos os termos, tendo assim maior similaridade com a consulta.

## Query 2

In [0]:
results_table2 = {
    'Binary similarity': get_bin_ranking('temperatura agora').index,
    'TF similarity': get_tf_ranking('temperatura agora').index,
    'TF IDF similarity': get_tf_idf_ranking('temperatura agora').index,
    'Bm25 similarity': get_bm25_ranking('temperatura agora').index
}
pd.DataFrame(results_table2)

,Binary similarity,TF similarity,TF IDF similarity,Bm25 similarity
0,137,165,137,137
1,0,184,77,77
2,112,101,136,136
3,128,124,165,165
4,129,139,184,184


O melhor resultado veio da similaridade calculada a partir do tf idf e bm25, uma vez que eles recuperarm os mesmos documentos, nos outros coeficientes, as noticias estavam relacionadas ao termo 'agora', no caso dessas duas, temos uma noticia relacionada a ondas de calor. Mesmo não sendo a resposta ideal, podemos supor que pode ser decorrente do nosso dataset não possuir notícias que discutem o tempo do dia.

## Query 3

In [0]:
results_table3 = {
    'Binary similarity': get_bin_ranking('nação brasileira').index,
    'TF similarity': get_tf_ranking('nação brasileira').index,
    'TF IDF similarity': get_tf_idf_ranking('nação brasileira').index,
    'Bm25 similarity': get_bm25_ranking('nação brasileira').index
}
pd.DataFrame(results_table3)

,Binary similarity,TF similarity,TF IDF similarity,Bm25 similarity
0,0,137,165,165
1,20,165,137,20
2,100,20,165,75
3,165,41,200,100
4,91,200,41,116


Assim como na query anterior, no mundo ideal, teríamos notícias muito relacionadas a patriotismo, problemas sociais barsileiros, mas os documentos recuperados também falam de política e economia, apesar de ter relação, a meu ponto de vista, não seria o que o usuário esperaria, o desempenho dos coeficientes, nesse caso, foi bastante semelhante, pois os documentos recuperados por cada um deles, eram muito similares

## Jacquard Index

In [0]:
def jacquard_index(first_data, second_data):
  intersection = set(first_data).intersection(set(second_data))
  union = set(first_data).union(set(second_data))
  return len(intersection)/len(union)

## Bin e TF

In [0]:
bin1 = get_bin_ranking('juíza federal').index
tf1 = get_tf_ranking('juíza federal').index
jacquard_index(bin1, tf1)

0.1111111111111111

In [0]:
bin2 = get_bin_ranking('temperatura agora').index
tf2 = get_tf_ranking('temperatura agora').index
jacquard_index(bin2, tf2)

0.0

In [0]:
bin3 = get_bin_ranking('nação brasileira').index
tf3 = get_tf_ranking('nação brasileira').index
jacquard_index(bin3, tf3)

0.25

## Bin e TF-IDF

In [0]:
bin4 = get_bin_ranking('juíza federal').index
tf_idf1 = get_tf_idf_ranking('juíza federal').index
jacquard_index(bin4, tf_idf1)

0.25

In [0]:
bin5 = get_bin_ranking('temperatura agora').index
tf_idf2 = get_tf_idf_ranking('temperatura agora').index
jacquard_index(bin5, tf_idf2)

0.1111111111111111

In [0]:
bin6 = get_bin_ranking('nação brasileira').index
tf_idf3 = get_tf_idf_ranking('nação brasileira').index
jacquard_index(bin6, tf_idf3)

0.125

## Bin e BM25

In [0]:
bin7 = get_bin_ranking('juíza federal').index
bm25_1 = get_bm25_ranking('juíza federal').index
jacquard_index(bin7, bm25_1)

0.25

In [0]:
bin8 = get_bin_ranking('temperatura agora').index
bm25_2 = get_bm25_ranking('temperatura agora').index
jacquard_index(bin8, bm25_2)

0.1111111111111111

In [0]:
bin9 = get_bin_ranking('nação brasileira').index
bm25_3 = get_bm25_ranking('nação brasileira').index
jacquard_index(bin9, bm25_3)

0.42857142857142855

## TF e TF-IDF

In [0]:
tf4 = get_tf_ranking('juíza federal').index
tf_idf4 = get_tf_idf_ranking('juíza federal').index
jacquard_index(tf4, tf_idf4)

0.6666666666666666

In [0]:
tf5 = get_tf_ranking('temperatura agora').index
tf_idf5 = get_tf_idf_ranking('temperatura agora').index
jacquard_index(tf5, tf_idf5)

0.25

In [0]:
tf6 = get_tf_ranking('nação brasileira').index
tf_idf6 = get_tf_idf_ranking('nação brasileira').index
jacquard_index(tf6, tf_idf6)

0.8

## TF e BM25

In [0]:
tf_6 = get_tf_ranking('juíza federal').index
bm25_6 = get_bm25_ranking('juíza federal').index
jacquard_index(bm25_6, tf_6)

0.6666666666666666

In [0]:
tf_7 = get_tf_ranking('temperatura agora').index
bm25_7 = get_bm25_ranking('temperatura agora').index
jacquard_index(bm25_7, tf_7)

0.25

In [0]:
tf_8 = get_tf_ranking('nação brasileira').index
bm25_8 = get_bm25_ranking('nação brasileira').index
jacquard_index(bm25_8, tf_8)

0.25

## TF-IDF e BM25

In [0]:
tf_idf7 = get_tf_idf_ranking('juíza federal').index
bm25_9 = get_bm25_ranking('juíza federal').index
jacquard_index(tf_idf7, bm25_9)

1.0

In [0]:
tf_idf8 = get_tf_idf_ranking('temperatura agora').index
bm25_10 = get_bm25_ranking('temperatura agora').index
jacquard_index(tf_idf8, bm25_10)

1.0

In [0]:
tf_idf9 = get_tf_idf_ranking('nação brasileira').index
bm25_11 = get_bm25_ranking('nação brasileira').index
jacquard_index(tf_idf9, bm25_11)

0.125